# Import Dependencies

In [3]:
import pandas as pd
import psycopg2
import os
from sqlalchemy import create_engine
from config import db_password
from config import db_name

# Read Video Games Data from CSV

In [5]:
# Set CSV file path
path = os.path.join('Output', 'Transformed_video_games_data.csv')